# Style Transfer with Deep Neural Networks


In this notebook, we’ll *recreate* a style transfer method that is outlined in the paper, [Image Style Transfer Using Convolutional Neural Networks, by Gatys](https://www.cv-foundation.org/openaccess/content_cvpr_2016/papers/Gatys_Image_Style_Transfer_CVPR_2016_paper.pdf) in PyTorch.

<img src='notebook_ims/vgg19_convlayers.png' width=80% />

In [4]:
!wget "http://cdn.masterstudies.com/element_db/37/3737_NUS-ISS.jpg" -O "./images/iss.jpg"

!wget "https://deepart-io.s3.amazonaws.com/img/style.jpg" -O "images/style.jpg"

--2019-01-19 03:04:39--  http://cdn.masterstudies.com/element_db/37/3737_NUS-ISS.jpg
Resolving cdn.masterstudies.com (cdn.masterstudies.com)... 205.185.216.42, 205.185.216.10
Connecting to cdn.masterstudies.com (cdn.masterstudies.com)|205.185.216.42|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 238902 (233K) [image/jpeg]
Saving to: ‘./images/iss.jpg’

./images/iss.jpg    100%[===================>] 233.30K  --.-KB/s    in 0.1s    

2019-01-19 03:04:43 (2.25 MB/s) - ‘./images/iss.jpg’ saved [238902/238902]

--2019-01-19 03:04:44--  https://deepart-io.s3.amazonaws.com/img/style.jpg
Resolving deepart-io.s3.amazonaws.com (deepart-io.s3.amazonaws.com)... 52.218.96.201
Connecting to deepart-io.s3.amazonaws.com (deepart-io.s3.amazonaws.com)|52.218.96.201|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 60111 (59K) [image/jpeg]
Saving to: ‘images/style.jpg’

images/style.jpg    100%[===================>]  58.70K  67.2KB/s    in 0.9s    

2019-

In [1]:
########
from style_transfer import load_image, transfer
from torchvision import models
import matplotlib.pyplot as plt
import torch
import torch.optim as optim
%matplotlib inline

target_img_path = 'images/iss.jpg'
style_img_path = 'images/style.jpg'

In [ ]:
device_name = "cuda" if torch.cuda.is_available() else "cpu"
device = torch.device(device_name)

# LOAD THE INPUT IMAGES:
# load in content and style image
content = load_image(target_img_path).to(device)
# Resize style to match content, makes code easier
style = load_image(style_img_path, shape=content.shape[-2:]).to(device)

# get the "features" portion of VGG19 (we will not need the "classifier" portion)
vgg = models.vgg19(pretrained=True).features

# freeze all VGG parameters since we're only optimizing the target image
for param in vgg.parameters():
    param.requires_grad_(False)

vgg.to(device)

# weights for each style layer 
# weighting earlier layers more will result in *larger* style artifacts
# notice we are excluding `conv4_2` our content representation
style_weights = {'conv1_1': 1.,
                 'conv2_1': 0.8,
                 'conv3_1': 0.5,
                 'conv4_1': 0.3,
                 'conv5_1': 0.1}

# you may choose to leave these as is
content_weight = 1  # alpha
style_weight = 1e6  # beta

# iteration hyperparameters
optimizer = optim.Adam
steps = 2000  # decide how many iterations to update your image (5000)

# for displaying the target image, intermittently
show_every = 400

target = transfer(
    device,
    style,
    content,
    vgg,
    content_weight,
    style_weight,
    style_weights,
    optimizer,
    steps,
)
        
# display content and final, target image
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(20, 10))
ax1.imshow(im_convert(style))
ax2.imshow(im_convert(content))
ax3.imshow(im_convert(target))

fig.save("./out/output.jpg")